In [ ]:
#see https://colab.research.google.com/drive/15LFRMVOfEiF__FswVqTQstZC5ocC6Ur0?usp=sharing for cropping functions, may require modification

# for each drone_survey:
#     for each row in ground truth:
#         crop rgb_tile from drone_survey
#         save tile to data/raster/tiles/drone_survey/idx.tif
